In [1]:
import pandas as pd

df1 = pd.read_csv('metrike/obrada_gotova/locust/1_prompt_test_results.csv')
df2 = pd.read_csv('metrike/obrada_gotova/locust/2_chat_test_results.csv')
df3 = pd.read_csv('metrike/obrada_gotova/locust/3_summarize_test_results.csv')
df4 = pd.read_csv('metrike/obrada_gotova/locust/4_batch_test_results.csv')

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

df.head(10)


,timestamp,test_name,prompt_index,model,prompt,ttft,tpot,tokens_per_sec,latency,prompt_tokens,output_tokens,output_text,status_code
0,2025-07-25 18:24:20.082775,single,0,mistral,Edit the following sentence to include the ter...,14.161550,0.091982,10.871688,14.165233,26.0,154,Edit the following sentence to include the ter...,200
1,2025-07-25 18:24:20.400474,single,0,meta-llama,Rewrite this story from the point of view of t...,10.752533,0.059760,16.733520,10.756852,52.0,180,Rewrite this story from the point of view of t...,200
2,2025-07-25 18:24:37.240732,single,0,mistral,Determine the capital of Peru.,31.326770,0.232081,4.308840,31.330938,7.0,135,Determine the capital of Peru.\n\nThe capital ...,200
3,2025-07-25 18:24:37.243646,single,0,mistral,Generate a fantasy creature with its unique sp...,31.323705,0.227031,4.404690,31.330242,10.0,138,Generate a fantasy creature with its unique sp...,200
4,2025-07-25 18:24:47.668964,single,0,mistral,Calculate the amount of time required for the ...,29.615530,0.197476,5.063902,29.621429,22.0,150,Calculate the amount of time required for the ...,200
5,2025-07-25 18:25:18.131661,single,0,meta-llama,Describe what dreams are.,39.833934,0.299588,3.337916,39.845215,5.0,133,Describe what dreams are. Dreams are a series ...,200
6,2025-07-25 18:25:18.135333,single,0,meta-llama,Sort the given items in order starting from th...,39.831499,0.265594,3.765144,39.839114,22.0,150,Sort the given items in order starting from th...,200
7,2025-07-25 18:25:20.590346,single,0,meta-llama,Create a unique greeting using only classic En...,18.024546,0.130692,7.651556,18.035548,10.0,138,Create a unique greeting using only classic En...,200
8,2025-07-25 18:25:26.653633,single,0,meta-llama,Calculate the total cost of a trip for two peo...,76.941881,0.437384,2.286321,76.979575,48.0,176,Calculate the total cost of a trip for two peo...,200
9,2025-07-25 18:25:40.110664,single,0,meta-llama,Convert this text into a headline. Apple unvei...,30.819117,0.208278,4.801273,30.825157,20.0,148,Convert this text into a headline. Apple unvei...,200


In [3]:
# remove latency outliers
def remove_latency_outliers(df, column='latency'):
    """
    Remove outliers from the specified latency column in the DataFrame.
    Outliers are defined as values greater than 3 standard deviations from the mean.
    """
    mean = df[column].mean()
    std_dev = df[column].std()
    threshold = mean + 3 * std_dev
    return df[df[column] <= threshold]
df = remove_latency_outliers(df, 'latency')

## Define the models of interest
models_to_analyze = ['gemma3', 'mistral', 'meta-llama']
test_names_to_analyze = ['single', 'chat', 'summarizer', 'batch']
# Filter the DataFrame to only include the specified models
df_filtered_models = df[
    (df['model'].isin(models_to_analyze)) &
    (df['test_name'].isin(test_names_to_analyze))
]

# Group by model and calculate the desired statistics for the 'ttft' column
statistics = df_filtered_models.groupby('model')['ttft'].agg(
    median='median',
    p99=lambda x: x.quantile(0.99),
    average='mean',
    std_dev='std',
    min='min',
    max='max',
).reset_index()

statistics



,model,median,p99,average,std_dev,min,max
0,meta-llama,23.567952,100.215025,29.500706,20.297717,10.304474,110.095600
1,mistral,18.431161,94.066306,25.025793,18.928881,6.177460,110.041632


In [4]:
# Filter the DataFrame to only include the specified models
df_filtered_models = df[
    (df['model'].isin(models_to_analyze)) &
    (df['test_name'].isin(test_names_to_analyze))
]

# Group by model and calculate the desired statistics for the 'ttft' column
statistics = df_filtered_models.groupby('test_name')['ttft'].agg(
    median='median',
    p99=lambda x: x.quantile(0.99),
    average='mean',
    std_dev='std',
    min='min',
    max='max',
).reset_index()

statistics


,test_name,median,p99,average,std_dev,min,max
0,batch,19.560757,33.439838,19.857469,5.123612,11.440661,33.780313
1,chat,81.949011,109.044336,76.477216,23.123458,31.040329,110.041632
2,single,44.799806,106.302854,49.834503,28.356571,6.177460,110.084522
3,summarizer,48.478897,99.484617,50.534812,26.441440,10.917340,110.095600


In [6]:
# Filter the DataFrame to only include the specified models
df_filtered_models = df[
    (df['model'].isin(models_to_analyze)) &
    (df['test_name'].isin(test_names_to_analyze))
]

# Group by model and calculate the desired statistics for the 'ttft' column
statistics = df_filtered_models.groupby('test_name')['tpot'].agg(
    median='median',
    p99=lambda x: x.quantile(0.99),
    average='mean',
    std_dev='std',
    min='min',
    max='max',
).reset_index()

statistics

,test_name,median,p99,average,std_dev,min,max
0,batch,0.141087,0.441852,0.153129,0.065900,0.074852,0.633125
1,chat,0.092380,0.173599,0.096191,0.034780,0.040119,0.208605
2,single,0.319870,0.822882,0.363909,0.225556,0.059760,1.998875
3,summarizer,0.003691,0.008165,0.003904,0.002137,0.000793,0.009032
